##Installation of the libraries

In [1]:
!pip install -q transformers

In [2]:
!pip install biopython

In [3]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

Torch optimization.

##All libraries needed for training

In [7]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2
import torch.nn.functional as F
# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re
import re
from Bio.PDB import PICIO, PDBIO
from Bio import PDB

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [9]:
def get_seqandang(file_path):
    file_model = file_path.replace(".cif","")
    pdbl = PDBList()
    #pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
    # import the needed class
    # instantiate the class to prepare the parser
    cif_parser = MMCIFParser()
    #structure = cif_parser.get_structure("3goe", "3goe.cif")
    structure = cif_parser.get_structure(file_model, file_path)
    model0 = structure[0]
    chain_A = model0['A']  # and we get chain A
    # dictionary converting 3-letter codes to 1-letter codes
    # this is a very common need in bioinformatics of proteins
    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    sequence = []
    for residue in chain_A:
        # for simplicity we can use X for heteroatoms (ions and water)
        sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
    print(''.join(sequence))

    structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

    chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

    ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

    d: Dict[Tuple[PDB.internal_coords.AtomKey,
                  PDB.internal_coords.AtomKey,
                  PDB.internal_coords.AtomKey,
                  PDB.internal_coords.AtomKey],
            PDB.internal_coords.Dihedron] = ic_chain.dihedra

    cnt = 1
    phi_angles = {}
    phi_angles_list = []
    psi_angles = {}
    psi_angles_list = []

    for key in d:
        if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
            phi_angles[key] = d[key].angle
            phi_angles_list.append(d[key].angle)
        elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
            psi_angles[key] = d[key].angle
            psi_angles_list.append(d[key].angle)


    structure.internal_to_atom_coordinates(verbose = True)
    io = PDBIO() #this is to write a pdb file again
    io.set_structure(structure)#set structure, the structure you wan tin the pdb file

    phi_angles_list.append(0)
    psi_angles_list.append(0)

    phi = np.asarray(phi_angles_list,dtype=np.float32)
    psi = np.asarray(psi_angles_list,dtype=np.float32)
    angles = torch.from_numpy(np.vstack((psi,phi)))

    seq_example =  ' '.join(sequence)
    return seq_example, angles

source_dir = "/Users/goudarzimandanagmail.com/Desktop/TransformerFromScratch/files"

for item in os.listdir(source_dir):
    item_path = os.path.join(source_dir, item)
    seq, angles = get_seqandang(item_path)
    print(f"Sequence: {seq}")
    print(f"Angles: {angles}")

MSVGKLTHYAIDLVLISVILAGIHRNTGLQFDTSHFSSVDFRRWFGNYLSFGESCYDKIVSMLKWSGYFKQNNLIFDYVNKEANKFIKEQTGRDLDDFKQKSS
Sequence: M S V G K L T H Y A I D L V L I S V I L A G I H R N T G L Q F D T S H F S S V D F R R W F G N Y L S F G E S C Y D K I V S M L K W S G Y F K Q N N L I F D Y V N K E A N K F I K E Q T G R D L D D F K Q K S S
Angles: tensor([[-160.6538,  179.2089,  177.5137,  177.1310,  178.1216,  175.7908,
          177.2993,  175.2542,  175.2995,  175.2201,  179.4641,  172.5769,
          176.2649,  176.7991,  176.1183,  179.2578,  174.2865,  175.9995,
          177.0388,  173.0741,  174.0014,  174.5040,  179.7067,  177.8972,
         -178.0160, -172.8403, -169.5418,  179.8021,  169.0070,  174.4699,
          179.6835,  176.8811, -178.4252,  174.1530,  176.8164, -176.5465,
         -172.9836,  174.6490,  179.2982,  177.1082,  175.5257,  178.4302,
          176.9164,  178.4984,  170.6663,  175.6387,  178.0272,  172.7495,
          176.4089,  171.6180,  173.7422,  175.4066,  176.5186,  174.375

In [ ]:
""" file_path = "AF-A0A1D8PCD7-F1-model_v4.cif"
file_model = "AF-A0A1D8PCD7-F1-model_v4"
pdbl = PDBList()
pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence)) """

Calculating the angles for the given sequence

In [ ]:
""" #phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file """

Putting angles in a matrix.

In [ ]:
""" phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)
angles = np.vstack((psi,phi)) """

##Embedding space creation (using Prot-Bert)

In [10]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq, return_tensors='pt')

/Users/goudarzimandanagmail.com/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


NameError: name 'seq_example' is not defined

##Single-head self unmasked attention layer

In [11]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

##Encoder with attention and 2 layer FFNN

In [12]:
class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output


In [13]:
class AngularLoss(nn.Module):
    def __init__(self):
        super(AngularLoss, self).__init__()

    def forward(self, predicted_angles, angles_tensor):
        predicted_angles_phi, predicted_angles_psi = predicted_angles[:, 0], predicted_angles[:, 1]
        angles_tensor_phi, angles_tensor_psi = angles_tensor[:, 0], angles_tensor[:, 1]

        predicted_angles_phi = (predicted_angles_phi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_phi = (angles_tensor_phi + torch.pi) % (2 * torch.pi) - torch.pi
        predicted_angles_psi = (predicted_angles_psi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_psi = (angles_tensor_psi + torch.pi) % (2 * torch.pi) - torch.pi

        difference_phi = torch.abs(predicted_angles_phi - angles_tensor_phi)
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

        difference_psi = torch.abs(predicted_angles_psi - angles_tensor_psi)
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))
        
        loss = loss_phi + loss_psi
        print(loss)
        return loss


In [14]:
""" class CustomLoss(nn.Module):
    def __init__(self, predicted_angles, angles_tensor):
        super(CustomLoss, self).__init__()
        self.predicted_angles= predicted_angles
        self.angles_tensor= angles_tensor
    def forward(self):
        print(self.predicted_angles)
        print(self.angles_tensor)
        d_list = []
        for i in range(len(self.angles_tensor)):
            x1, y1 = self.predicted_angles[0][i]
            x2, y2 = self.angles_tensor[i]
            ax_x = torch.min(torch.abs(x2 - x1), torch.abs(360 - torch.abs(x2 - x1)))
            ax_y = torch.min(torch.abs(y2 - y1), torch.abs(360 - torch.abs(y2 - y1)))
            d = torch.sqrt(ax_x**2 + ax_y**2)
            d_list.append(d)
        d_tensor = torch.stack(d_list)
        return d_tensor.mean() """
 

' class CustomLoss(nn.Module):\n    def __init__(self, predicted_angles, angles_tensor):\n        super(CustomLoss, self).__init__()\n        self.predicted_angles= predicted_angles\n        self.angles_tensor= angles_tensor\n    def forward(self):\n        print(self.predicted_angles)\n        print(self.angles_tensor)\n        d_list = []\n        for i in range(len(self.angles_tensor)):\n            x1, y1 = self.predicted_angles[0][i]\n            x2, y2 = self.angles_tensor[i]\n            ax_x = torch.min(torch.abs(x2 - x1), torch.abs(360 - torch.abs(x2 - x1)))\n            ax_y = torch.min(torch.abs(y2 - y1), torch.abs(360 - torch.abs(y2 - y1)))\n            d = torch.sqrt(ax_x**2 + ax_y**2)\n            d_list.append(d)\n        d_tensor = torch.stack(d_list)\n        return d_tensor.mean() '

In [15]:
class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            #loss = CustomLoss()
            #loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())
            #print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss



In [16]:
""" def inc_lr(step_size,init_lr,epoch):
    if(epoch % step_size):
        return init_lr + 0.05 """

' def inc_lr(step_size,init_lr,epoch):\n    if(epoch % step_size):\n        return init_lr + 0.05 '

In [33]:
from transformers import BertModel, BertTokenizer
import re
source_dir = "/Users/goudarzimandanagmail.com/Desktop/TransformerFromScratch/files"
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 100
dropout_rate = 0.1
model = TransformerModel(embed_dim=1024, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2, dropout_rate = dropout_rate)
criterion = AngularLoss()

bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
for item in os.listdir(source_dir):
    item_path = os.path.join(source_dir, item)
    print(item_path)
    seq_example, angles = get_seqandang(item_path)
    seq = re.sub(r"[UZOB]", "X", seq_example)
    encoded_input = tokenizer(seq, return_tensors='pt')
    output = bert_model(**encoded_input)
    embedded_pb = output.last_hidden_state
    N, D = embedded_pb.size()[1], embedded_pb.size()[2]
    embeddings = embedded_pb
    angles_tensor = (angles.T)*(np.pi/180)
    trainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles_tensor)
    trainer.train()

/Users/goudarzimandanagmail.com/Desktop/TransformerFromScratch/files/AF-A0A1D8PDZ3-F1-model_v4.cif
MSVGKLTHYAIDLVLISVILAGIHRNTGLQFDTSHFSSVDFRRWFGNYLSFGESCYDKIVSMLKWSGYFKQNNLIFDYVNKEANKFIKEQTGRDLDDFKQKSS
tensor(3.9215, grad_fn=<AddBackward0>)
tensor(2.8768, grad_fn=<AddBackward0>)
tensor(0.9355, grad_fn=<AddBackward0>)
tensor(2.8946, grad_fn=<AddBackward0>)
tensor(2.1416, grad_fn=<AddBackward0>)
tensor(0.7607, grad_fn=<AddBackward0>)
tensor(1.5024, grad_fn=<AddBackward0>)
tensor(1.5271, grad_fn=<AddBackward0>)
tensor(1.2096, grad_fn=<AddBackward0>)
tensor(0.8546, grad_fn=<AddBackward0>)
tensor(0.8899, grad_fn=<AddBackward0>)
tensor(0.8899, grad_fn=<AddBackward0>)
tensor(0.9641, grad_fn=<AddBackward0>)
tensor(0.9285, grad_fn=<AddBackward0>)
tensor(0.8348, grad_fn=<AddBackward0>)
tensor(0.7844, grad_fn=<AddBackward0>)
tensor(0.7813, grad_fn=<AddBackward0>)
tensor(0.8182, grad_fn=<AddBackward0>)
tensor(0.8046, grad_fn=<AddBackward0>)
tensor(0.8010, grad_fn=<AddBackward0>)
tensor(0.7465, gr

In [25]:
seq_example

'M Y T T L E Y I C T I V S I I N L I T A L V I Y I I D R K Q G V S I N S G K H F Q S F K T C I T M S I L F G V L S M C V T L N N L H H S H R I D Q'

In [34]:
angles_tensor = angles.T*(np.pi/180)
predicted_angles = model.forward(embedded_pb)[:, :len(angles_tensor)]

In [ ]:
""" pred = predicted_angles*(180/np.pi)
torch.save(pred,'predictions/GELU_dr_A0A1D8PD42-F1_200_dyn001_AdamW.pt') """

In [35]:
predicted_angles

tensor([[[ 3.1051, -0.7658],
         [ 2.9325, -0.7613],
         [ 3.1535, -0.6937],
         [ 2.9358, -0.6953],
         [ 2.9183, -0.6007],
         [ 3.0147, -0.7718],
         [ 2.9615, -0.4045],
         [ 2.9006, -0.6771],
         [ 3.0558, -0.6476],
         [ 2.9748, -0.7298],
         [ 3.1507, -0.6578],
         [ 3.0861, -0.5536],
         [ 3.0323, -0.7441],
         [ 2.9661, -0.7752],
         [ 3.0079, -0.7179],
         [ 2.9052, -0.6508],
         [ 2.9618, -0.7329],
         [ 2.9748, -0.6075],
         [ 3.0610, -0.7654],
         [ 3.1330, -0.6901],
         [ 3.1293, -0.6808],
         [ 3.1401, -0.7213],
         [ 2.9964, -0.6852],
         [ 2.9243, -0.6907],
         [ 3.1513, -0.6105],
         [ 3.0310, -0.7767],
         [ 3.0368, -0.6510],
         [ 3.0414, -0.7703],
         [ 2.9772, -0.7723],
         [ 3.1547, -0.6609],
         [ 3.0628, -0.6396],
         [ 3.0074, -0.7581],
         [ 2.9218, -0.6817],
         [ 3.0690, -0.7047],
         [ 3.0

In [37]:
angles_tensor

tensor([[ 3.0262,  2.2452],
        [ 3.0642,  2.1477],
        [ 3.0746,  2.5996],
        [ 3.1220, -0.5960],
        [ 3.0463, -0.7632],
        [ 3.0951, -0.7196],
        [ 3.0892, -0.8122],
        [ 3.1167, -0.8545],
        [ 3.1394, -0.7313],
        [ 3.0750, -0.7737],
        [ 3.0963, -0.8113],
        [ 3.0612, -0.7020],
        [ 3.1040, -0.7594],
        [ 3.1156, -0.7797],
        [ 3.0814, -0.8392],
        [ 3.1243, -0.6441],
        [ 3.0622, -0.7325],
        [ 3.0784, -0.7900],
        [ 3.0477, -0.7584],
        [ 3.0911, -0.7604],
        [ 3.1001, -0.7638],
        [ 3.0756, -0.8096],
        [ 3.1014, -0.8086],
        [ 3.1237, -0.7878],
        [ 3.1198, -0.7124],
        [-3.1371, -0.8883],
        [ 3.0814, -0.7226],
        [ 3.0609, -0.6543],
        [ 3.1093, -0.5546],
        [-3.0384, -0.0390],
        [ 3.1400,  0.2224],
        [ 3.0385,  2.2462],
        [ 3.1095,  2.1843],
        [-3.0992, -0.4820],
        [ 3.0979, -0.1280],
        [-3.1350,  0

In [ ]:
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object
#if you modify this, you will modify the orgiginal

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 0
phi_angles = {}
psi_angles = {}

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':

        d[key].angle = angles_tensor[cnt, 0].item()
        #print('phi clculated')
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        d[key].angle = angles_tensor[cnt, 1].item()
        #print("psi calcukated")
        #print(cnt)
        cnt += 1



structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('AF-A0A1D8PD42-F1-model_v4_pred.pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering

In [ ]:
angles_tensor

tensor([[ 173.3910,  128.6415],
        [ 175.5637,  123.0513],
        [ 176.1591,  148.9463],
        [ 178.8801,  -34.1475],
        [ 174.5416,  -43.7297],
        [ 177.3356,  -41.2293],
        [ 176.9977,  -46.5382],
        [ 178.5731,  -48.9617],
        [ 179.8747,  -41.9026],
        [ 176.1833,  -44.3280],
        [ 177.4022,  -46.4853],
        [ 175.3916,  -40.2197],
        [ 177.8444,  -43.5102],
        [ 178.5081,  -44.6731],
        [ 176.5513,  -48.0830],
        [ 179.0103,  -36.9032],
        [ 175.4486,  -41.9671],
        [ 176.3769,  -45.2628],
        [ 174.6190,  -43.4555],
        [ 177.1076,  -43.5659],
        [ 177.6247,  -43.7630],
        [ 176.2167,  -46.3874],
        [ 177.6991,  -46.3290],
        [ 178.9726,  -45.1391],
        [ 178.7514,  -40.8169],
        [-179.7433,  -50.8967],
        [ 176.5511,  -41.4023],
        [ 175.3784,  -37.4864],
        [ 178.1508,  -31.7786],
        [-174.0891,   -2.2330],
        [ 179.9095,   12.7426],
        